In [2]:
import os
import pandas as pd

In [3]:
#Import files
genfile = "../rawData/EIA_generation_by_state.xls"
emissfile = "../rawData/EIA_emissions_by_state.xls"
greenhousefile = "../rawData/ghgp_data_by_year.xls"

In [4]:
#Format into pandas DF
generationXls = pd.ExcelFile(genfile)
emissfileXls = pd.ExcelFile(emissfile)
greenhouseXls = pd.ExcelFile(greenhousefile)

generation_df = generationXls.parse('Net_Generation_1990-2019 Final',skiprows=1,header=0)
emissions_df = emissfileXls.parse('State Emissions',header=0)
greenhouse_df = greenhouseXls.parse('Direct Emitters',skiprows=3,header=0)

In [5]:
# generation_df.columns
# emissions_df.head()
# greenhouse_df.head()           
# greenhouse_df.columns

In [6]:
#Format columns & clean data
generation_df.columns= ['year','state','producer type','energy source','generation(mwH)']
generation_df = generation_df.replace({'state':{'US-Total':'US', 'US-TOTAL': 'US'},
                                       'energy source':{'All Sources':'Total'}})
# generation_df.head()

emissions_df.columns = ['year','state','producer type','energy source','CO2 (MT)','SO2 (MT)','NOx (MT)']
emissions_df = emissions_df.replace({'state':{'US-Total':'US','US-TOTAL':'US'},
                                     'energy source':{'All Sources':'Total'}})
# emissions_df.head()

greenhouse_df.columns = ['facility id','frs id','facility name','city','state','zip','address','county','latitude','longitude',
                         'naics code','industry type (subpart)','industry type (sectors)',
                         '2019 Total Emissions','2018 Total Emissions','2017 Total Emissions','2016 Total Emissions','2015 Total Emissions',
                         '2014 Total Emissions','2013 Total Emissions','2012 Total Emissions','2011 Total Emissions']

# greenhouse_df.head()

In [7]:
#Pulling in one producer type for analysis with MERGE
Gen_by_state = generation_df.loc[generation_df['producer type'] == 'Total Electric Power Industry']
# Gen_by_state.head()

Emiss_by_state = emissions_df.loc[emissions_df['producer type'] == 'Total Electric Power Industry']

stateEnergy = pd.merge(Emiss_by_state,Gen_by_state,
                       on=['year','state','producer type','energy source'],how='outer')
# stateEnergy.head()

greenhouse_df_loc = greenhouse_df.loc[greenhouse_df['industry type (sectors)'] == 'Power Plants']
greenhouse_update = greenhouse_df_loc[~greenhouse_df['state'].isin(['VI,GU,PR'])]

C:\Users\cupan\.conda\envs\pythondata\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


In [14]:
# greenhouse_update.head()
# greenhouse_df_loc
greenhouse_update.head()
# greenhouse_update.values

,facility id,frs id,facility name,city,state,zip,address,county,latitude,longitude,...,industry type (sectors),2019 Total Emissions,2018 Total Emissions,2017 Total Emissions,2016 Total Emissions,2015 Total Emissions,2014 Total Emissions,2013 Total Emissions,2012 Total Emissions,2011 Total Emissions
4,1000112,1.100198e+11,23rd and 3rd,BROOKLYN,NY,11232,730 3rd Avenue,Kings,40.6630,-74.0000,...,Power Plants,44658.384,70705.288,52698.656,66797.352,67399.252,40722.200,102599.220,68984.744,100591.828
24,1001106,1.100409e+11,48th Street Peaking Station,HOLLAND,MI,49423,491 E 48TH ST,Allegan,42.7553,-86.0853,...,Power Plants,13841.000,32159.832,50999.760,NaN,12264.256,12516.852,9659.064,29574.240,9814.756
27,1000766,1.100193e+11,59th Street,NEW YORK,NY,10019,850 12TH AVE,New York,40.7711,-73.9911,...,Power Plants,224910.106,260405.936,218874.796,232873.708,238778.194,262718.648,206935.342,199034.550,265415.758
29,1000767,1.100193e+11,74th Street,NEW YORK,NY,10021,506 E 75TH ST,New York,40.7680,-73.9515,...,Power Plants,254385.114,270795.330,194944.188,205696.230,255020.780,272983.418,226196.740,148328.410,297071.804
30,1001033,1.100449e+11,A B Brown Generating Station,MOUNT VERNON,IN,47620,8511 WELBORN ROAD,Posey,37.9053,-87.7150,...,Power Plants,2789540.744,2741626.714,2172917.248,2315093.224,2663665.556,3143355.920,2556867.744,2538211.824,2270617.680


In [9]:
#FORMAT GREENHOUSE
greenhouse_groupby = greenhouse_update.groupby(['state'])['2019 Total Emissions', '2018 Total Emissions', '2017 Total Emissions',
       '2016 Total Emissions', '2015 Total Emissions', '2014 Total Emissions',
       '2013 Total Emissions', '2012 Total Emissions', '2011 Total Emissions'].sum()
greenhouse_groupby.columns=['2019','2018','2017','2016','2015','2014','2013','2012','2011']
greenhouse_groupby.head()
idgreenhouse = greenhouse_groupby.stack()
greenhouse_by_state = pd.DataFrame(idgreenhouse)
greenhouse_by_state.reset_index(inplace=True)
greenhouse_by_state.columns = ['state','year','greenhouse emissions']
greenhouse_by_state['year'] = pd.to_numeric(greenhouse_by_state['year'])
# greenhouse_by_state.head()
# greenhouse_by_state.dtypes

C:\Users\cupan\.conda\envs\pythondata\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [10]:
stateEnergy.to_csv('../cleanData/stateEnergy.csv',index=False,header=True)
greenhouse_by_state.to_csv('../cleanData/greenhouse_by_state.csv',index=False,header=True)